In [32]:

%run utils.ipynb
#Leemos el dataset reducido
es_train, es_train, es_test, en_test,es_texts,en_texts = init_data()


#Ahora vamos a tokenizar los diferentes datasets para ello usaremos Keras y su Tokenizer
es_tok,en_tok,es_len,en_len,es_vocab,en_vocab = init_tokens(en_texts,es_texts)

print("Español- Numero de palabras: ", es_vocab, "\t Frase mas larga: ", es_len)
print("Ingles - Numero de palabras: ", en_vocab, "\t Frase mas larga: ", en_len)

#Dividimos el dataset en entrenamiento y prueba
es_train, es_test, en_train, en_test = split_dataset(es_texts,en_texts)



Español- Numero de palabras:  4050 	 Frase mas larga:  102
Ingles - Numero de palabras:  4841 	 Frase mas larga:  105


In [33]:
%run utils.ipynb
from tensorflow.keras.models import Model

model = create_model_inference_train(es_len,en_len,es_vocab,en_vocab)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["acc"])
model.summary()


Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_24 (InputLayer)           [(None, 102, 4050)]  0                                            
__________________________________________________________________________________________________
input_25 (InputLayer)           [(None, 104, 4841)]  0                                            
__________________________________________________________________________________________________
gru_17 (GRU)                    [(None, 128), (None, 1605120     input_24[0][0]                   
__________________________________________________________________________________________________
gru_18 (GRU)                    (None, 104, 128)     1908864     input_25[0][0]                   
                                                                 gru_17[0][1]              

In [34]:
%run utils.ipynb

#Procesaremos las frases para nuestro modelo que tendra 2 "inputs" y un "output".
#Las entradas son: El one hot vector de las palabras en español y el onehot vector en ingles menos la ultima palabra
#La salida es el one hot vector en ingles menos la primera palabra

train_inference_model()

Epoch 1/5
7/7 [==============================] - 72s 8s/step - loss: 8.4663 - acc: 0.4247 - val_loss: 8.3848 - val_acc: 0.9837
Epoch 2/5
7/7 [==============================] - 65s 9s/step - loss: 8.3378 - acc: 0.9640 - val_loss: 8.0294 - val_acc: 0.8004
Epoch 3/5
7/7 [==============================] - 65s 9s/step - loss: 7.5181 - acc: 0.7983 - val_loss: 4.5889 - val_acc: 0.7825
Epoch 4/5
7/7 [==============================] - 55s 8s/step - loss: 3.8621 - acc: 0.7845 - val_loss: 1.9586 - val_acc: 0.7731
Epoch 5/5
7/7 [==============================] - 48s 7s/step - loss: 1.6167 - acc: 0.7754 - val_loss: 0.8729 - val_acc: 0.7731


Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        [(None, 119, 4937)]       0         
_________________________________________________________________
gru_15 (GRU)                 [(None, 128), (None, 128) 1945728   
Total params: 1,945,728
Trainable params: 1,945,728
Non-trainable params: 0
_________________________________________________________________
Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_22 (InputLayer)           [(None, 1, 4161)]    0                                            
__________________________________________________________________________________________________
input_23 (InputLayer)           [(None, 128)]        0                                          

In [23]:
from tensorflow.keras import layers as layers
import tensorflow.keras.models as models
import numpy as np

def word_to_onehot(tokenizer, word, vocab_size):
    seq = tokenizer.texts_to_sequences([[word]])
    seq = to_categorical(seq, num_classes=vocab_size)
    seq = np.expand_dims(seq, axis=1)
    return seq

def probability_to_word(tokenizer, prob):
    word_index = np.argmax(prob[0,:], axis=-1)
    word = tokenizer.index_word[word_index]
    return word
#Ahora lo siguiente que tenemos que hacer es cargar el modelo entrenado ("Modelo anterior a este") y copiar
#los pesos
#En nuestro nuevo modelo hay 3 capas que contienen pesos, la Gru layer del encoder y decoder y la Dense layer del 
#decoder
#Leer archivo


#Introducimos una frase
es_sentence=["de acuerdo con lo que se. "]

es_seq = sentences_to_sequences(es_tok,es_sentence,es_len, es_vocab)
#Vamos a predecir el primer el estado a traves del encoder
en_state = encoder.predict(es_seq)
en_seq = word_to_onehot(en_tok, "sos", en_vocab)
en_sentence = ""
for i in range(en_len):
    en_probability, en_state = decoder.predict([en_seq,en_state])
    word = probability_to_word(en_tok,en_probability)
    en_seq = word_to_onehot(en_tok, word, en_vocab)
    if word == "eos": break
    en_sentence += word+" "
    print(word)
print(en_sentence)

KeyError: 0